In [1]:
"""
Dedalus script simulating 2D horizontally-periodic Rayleigh-Benard convection.
This script demonstrates solving a 2D Cartesian initial value problem. It can
be ran serially or in parallel, and uses the built-in analysis framework to save
data snapshots to HDF5 files. The `plot_snapshots.py` script can be used to
produce plots from the saved data. It should take about 5 cpu-minutes to run.

For incompressible hydro with two boundaries, we need two tau terms for each the
velocity and buoyancy. Here we choose to use a first-order formulation, putting
one tau term each on auxiliary first-order gradient variables and the others in
the PDE, and lifting them all to the first derivative basis. This formulation puts
a tau term in the divergence constraint, as required for this geometry.

To run and plot using e.g. 4 processes:
    $ mpiexec -n 4 python3 rayleigh_benard.py
    $ mpiexec -n 4 python3 plot_snapshots.py snapshots/*.h5
"""


'\nDedalus script simulating 2D horizontally-periodic Rayleigh-Benard convection.\nThis script demonstrates solving a 2D Cartesian initial value problem. It can\nbe ran serially or in parallel, and uses the built-in analysis framework to save\ndata snapshots to HDF5 files. The `plot_snapshots.py` script can be used to\nproduce plots from the saved data. It should take about 5 cpu-minutes to run.\n\nThe problem is non-dimensionalized using the box height and freefall time, so\nthe resulting thermal diffusivity and viscosity are related to the Prandtl\nand Rayleigh numbers as:\n    kappa = (Rayleigh * Prandtl)**(-1/2)\n    nu = (Rayleigh / Prandtl)**(-1/2)\n\nFor incompressible hydro with two boundaries, we need two tau terms for each the\nvelocity and buoyancy. Here we choose to use a first-order formulation, putting\none tau term each on auxiliary first-order gradient variables and the others in\nthe PDE, and lifting them all to the first derivative basis. This formulation puts\na tau 

In [1]:
import numpy as np
import dedalus.public as d3
import logging
logger = logging.getLogger(__name__)

In [2]:
# Parameters
Lx, Lz = 4,1
Nx, Nz = 256, 64
Ra_D= -1.24e5
D_0 = 0
D_H = 1
M_0 = 0
M_H = -1
N_s2=2

Prandtl = 0.7
dealias = 3/2
stop_sim_time = 200
timestepper = d3.RK222
max_timestep = 0.125
dtype = np.float64

In [3]:
# Bases
coords = d3.CartesianCoordinates('x','z')
dist = d3.Distributor(coords, dtype=dtype)
xbasis = d3.RealFourier(coords['x'], size=Nx, bounds=(0, Lx), dealias=dealias)
zbasis = d3.ChebyshevT(coords['z'], size=Nz, bounds=(0, Lz), dealias=dealias)

In [6]:
# Fields
p = dist.Field(name='p', bases=(xbasis,zbasis))
D = dist.Field(name='D', bases=(xbasis,zbasis))
M = dist.Field(name='M', bases=(xbasis,zbasis))
u = dist.VectorField(coords, name='u', bases=(xbasis,zbasis))
Z = dist.Field(name='Z', bases=zbasis)
tau_p = dist.Field(name='tau_p')
tau_B1 = dist.Field(name='tau_B1', bases=xbasis)
tau_B2 = dist.Field(name='tau_B2', bases=xbasis)
tau_D1 = dist.Field(name='tau_D1', bases=xbasis)
tau_D2 = dist.Field(name='tau_D2', bases=xbasis)
tau_M1 = dist.Field(name='tau_M1', bases=xbasis)
tau_M2 = dist.Field(name='tau_M2', bases=xbasis)
tau_u1 = dist.VectorField(coords, name='tau_u1', bases=xbasis)
tau_u2 = dist.VectorField(coords, name='tau_u2', bases=xbasis)


# Substitutions

kappa = (Ra_D * Prandtl/((D_0-D_H)*Lz**3))**(-1/2)
nu = (Ra_D / (Prandtl*(D_0-D_H)*Lz**3))**(-1/2)
      
#Kuo_Bretherton Equilibrium
Ra_M = Ra_D*(M_0-M_H)/(D_0-D_H)
G_D=(D_0-D_H)/Lz
G_M=(M_0-M_H)/Lz
Ra_BV=N_s2*Lz**4/(nu*kappa)
print(Ra_M)
print(Ra_BV)

x,z = dist.local_grids(xbasis,zbasis)
Z['g']=z
Z.change_scales(3/2)

ex,ez = coords.unit_vector_fields(dist)
lift_basis = zbasis.derivative_basis(1)
lift = lambda A: d3.Lift(A, lift_basis, -1)

B_op = (np.absolute(D - M - N_s2*Z)+ M + D - N_s2*Z)/2

Max = lambda A,B: (abs(A-N_s2*Z-B)+A-N_s2*Z+B)/2
eva = lambda A: A.evaluate()

dz= lambda A: d3.Differentiate(A, coords['z'])
dx= lambda A: d3.Differentiate(A, coords['x'])

ux=u@ex
uz=u@ez

grad_u = d3.grad(u) + ez*lift(tau_u1) # First-order reduction
grad_M = d3.grad(M) + ez*lift(tau_M1) # First-order reduction
grad_D = d3.grad(D) + ez*lift(tau_D1) # First-order reduction

124000.0
248000.00000000003


In [8]:
# Problem
# First-order form: "div(f)" becomes "trace(grad_f)"
# First-order form: "lap(f)" becomes "div(grad_f)"
problem = d3.IVP([p, M, D, u, tau_p, tau_M1, tau_M2, tau_D1, tau_D2, tau_u1, tau_u2], namespace=locals())
problem.add_equation("trace(grad_u) + tau_p= 0")
problem.add_equation("dt(M) - kappa*div(grad_M) + lift(tau_M2) - G_M*uz= - u@grad(M)")
problem.add_equation("dt(D) - kappa*div(grad_D) + lift(tau_D2) - G_D*uz= - u@grad(D)")
problem.add_equation("dt(u) - nu*div(grad_u) + grad(p)  + lift(tau_u2) = - u@grad(u)+ B_op*ez")
problem.add_equation("M(z=0) = M_0")
problem.add_equation("D(z=0) = D_0")
problem.add_equation("uz(z=0)= 0")
problem.add_equation("uz(z=Lz)= 0")
problem.add_equation("dz(ux)(z=0)=0")
problem.add_equation("dz(ux)(z=Lz)=0")
problem.add_equation("M(z=Lz) = M_H")
problem.add_equation("D(z=Lz) = D_H")
problem.add_equation("integ(p) = 0") # Pressure gauge

{'LHS': Integrate(Integrate(<Field 22389128265840>)),
 'RHS': 0,
 'condition': 'True',
 'tensorsig': (),
 'dtype': numpy.float64,
 'M': 0,
 'L': Integrate(Integrate(<Field 22389128265840>)),
 'F': <Field 22389125810496>,
 'domain': <dedalus.core.domain.Domain at 0x145d0da01c30>,
 'matrix_dependence': array([ True,  True]),
 'matrix_coupling': array([False,  True])}

In [9]:
# Solver
solver = problem.build_solver(timestepper)
solver.stop_sim_time = stop_sim_time


2023-07-11 15:11:27,093 subsystems 0/1 INFO :: Building subproblem matrices 1/128 (~1%) Elapsed: 1s, Remaining: 1m 49s, Rate: 1.2e+00/s
2023-07-11 15:11:28,095 subsystems 0/1 INFO :: Building subproblem matrices 13/128 (~10%) Elapsed: 2s, Remaining: 16s, Rate: 7.0e+00/s
2023-07-11 15:11:29,546 subsystems 0/1 INFO :: Building subproblem matrices 26/128 (~20%) Elapsed: 3s, Remaining: 13s, Rate: 7.8e+00/s
2023-07-11 15:11:31,219 subsystems 0/1 INFO :: Building subproblem matrices 39/128 (~30%) Elapsed: 5s, Remaining: 11s, Rate: 7.8e+00/s
2023-07-11 15:11:32,653 subsystems 0/1 INFO :: Building subproblem matrices 52/128 (~41%) Elapsed: 6s, Remaining: 9s, Rate: 8.1e+00/s
2023-07-11 15:11:34,286 subsystems 0/1 INFO :: Building subproblem matrices 65/128 (~51%) Elapsed: 8s, Remaining: 8s, Rate: 8.1e+00/s
2023-07-11 15:11:36,139 subsystems 0/1 INFO :: Building subproblem matrices 78/128 (~61%) Elapsed: 10s, Remaining: 6s, Rate: 7.9e+00/s
2023-07-11 15:11:36,275 subsystems 0/1 INFO :: Building 

In [10]:
# Initial condition
D.fill_random('g', seed=42, distribution='normal', scale=1e-3) # Random noise
D['g'] *= z * (Lz - z) # Damp noise at walls
D['g'] += (D_H-D_0)*z # Add linear background
M.fill_random('g', seed=28, distribution='normal', scale=1e-3) # Random noise
M['g'] *= z * (Lz - z) # Damp noise at walls
M['g'] += (M_H-M_0)*z # Add linear background

In [11]:
# Analysis
snapshots = solver.evaluator.add_file_handler('snapshots', sim_dt=0.25, max_writes=50)
snapshots.add_task(M, name='moist buoyancy')
snapshots.add_task(D, name='dry buoyancy')
snapshots.add_task(-d3.div(d3.skew(u)), name='vorticity')

In [12]:
# CFL
CFL = d3.CFL(solver, initial_dt=max_timestep, cadence=10, safety=0.5, threshold=0.05,
             max_change=1.5, min_change=0.5, max_dt=max_timestep)
CFL.add_velocity(u)

In [13]:
# Flow properties
flow = d3.GlobalFlowProperty(solver, cadence=10)
flow.add_property(np.sqrt(u@u)/nu, name='Re')


In [14]:
# Main loop
startup_iter = 10
try:
    logger.info('Starting main loop')
    while solver.proceed:
        timestep = CFL.compute_timestep()
        solver.step(timestep)
        if (solver.iteration-1) % 10 == 0:
            max_Re = flow.max('Re')
            logger.info('Iteration=%i, Time=%e, dt=%e, max(Re)=%f' %(solver.iteration, solver.sim_time, timestep, max_Re))
except:
    logger.error('Exception raised, triggering end of main loop.')
    raise
finally:
    solver.log_stats()

2023-07-11 15:12:56,405 __main__ 0/1 INFO :: Starting main loop
2023-07-11 15:12:57,734 __main__ 0/1 INFO :: Iteration=1, Time=1.250000e-01, dt=1.250000e-01, max(Re)=0.000000
2023-07-11 15:12:58,582 __main__ 0/1 INFO :: Iteration=11, Time=1.375000e+00, dt=1.250000e-01, max(Re)=0.009011
2023-07-11 15:12:59,426 __main__ 0/1 INFO :: Iteration=21, Time=2.625000e+00, dt=1.250000e-01, max(Re)=0.018408
2023-07-11 15:13:00,265 __main__ 0/1 INFO :: Iteration=31, Time=3.875000e+00, dt=1.250000e-01, max(Re)=0.045107
2023-07-11 15:13:01,116 __main__ 0/1 INFO :: Iteration=41, Time=5.125000e+00, dt=1.250000e-01, max(Re)=0.117608
2023-07-11 15:13:01,959 __main__ 0/1 INFO :: Iteration=51, Time=6.375000e+00, dt=1.250000e-01, max(Re)=0.312292
2023-07-11 15:13:02,798 __main__ 0/1 INFO :: Iteration=61, Time=7.625000e+00, dt=1.250000e-01, max(Re)=0.833659
2023-07-11 15:13:03,610 __main__ 0/1 INFO :: Iteration=71, Time=8.875000e+00, dt=1.250000e-01, max(Re)=2.243625
2023-07-11 15:13:04,398 __main__ 0/1 INFO